## データとライブラリの読み込み

In [1]:
import pandas as pd
import pandas_profiling
import numpy as np

#google drive読み込み
# from google.colab import drive, files
# drive.mount('/content/drive')
# dataPath = "/content/drive/My Drive/01 Lab/novel_bookmark/datasets/"
# test = pd.read_csv(dataPath + 'test.csv')
# train = pd.read_csv(dataPath +'train.csv')
# sub = pd.read_csv(dataPath +'sample_submission.csv')

#local 読み込み
dataPath = './datasets/'
titlePath = './titledata/'

test = pd.read_csv(dataPath + 'test.csv')
train = pd.read_csv(dataPath +'train.csv')
sub = pd.read_csv(dataPath +'sample_submission.csv')

## データの可視化、確認

In [12]:
#データの可視化
# !pip install pandas-profiling
# !pip install --upgrade pandas_profiling
# train.profile_report()

In [13]:
# df = train
# print(len(df[train['fav_novel_cnt_bin']==4]))

# sub = df[train['fav_novel_cnt_bin']==4]
# sub.to_csv('pre_file.csv', index=False)
# files.download('pre_file.csv')

## 特徴量エンジニアリング

In [ ]:
test = pd.read_csv(dataPath + 'test.csv')
train = pd.read_csv(dataPath +'train.csv')
sub = pd.read_csv(dataPath +'sample_submission.csv')

data = pd.concat([train, test], sort=False)#データの統合

#必要のないカラムを削除
#ncode:小説コード, writer, userid, pc_or_k:小説の投稿機種 
delete_columns = ['ncode','writer', 'userid', 'pc_or_k']
data.drop(delete_columns, axis=1, inplace=True)

#novel_type (1, 2)を(0, 1)に変更
data['novel_type'] = data['novel_type'].map({1: 0, 2: 1})

#'general_firstup'(初回投稿日)を今日までの日数に変換
import datetime
def calc_diff_days(x):#return days(int)
  y_m_d = x.split(" ")[0]
  y, m, d = map(int, y_m_d.split("-"))
  ymd = datetime.datetime(y, m, d)
  diff = datetime.datetime.today() - ymd
  return diff.days
data['general_firstup'] = data['general_firstup'].map(calc_diff_days)

#一時的にテキストデータを削除
text_columns = ['title','story', 'keyword']
data.drop(text_columns, axis=1, inplace=True)

#titleデータをベクトルに変換
title_data = pd.read_csv(titlePath+'title_vec.csv', index_col=0)

data = title_data

#dataをtest, trainに分割
train = data[:len(train)]
test = data[len(train):]

train.to_csv('./tmp/tmp_train.csv')

data

In [ ]:
sentence_csv = pd.read_csv(titlePath+'sentenceVec.csv', index_col=0)
sentence_vec_data = sentence_csv['sentenceVec']
tmp = np.array(sentence_vec_data)
tmp

### 訓練データと答えに分割

In [6]:
x_train = train.drop('fav_novel_cnt_bin', axis=1)#訓練データ
y_train = train['fav_novel_cnt_bin']#訓練データの答え
x_test = test#テストデータ

x_train = train[:35000].drop('fav_novel_cnt_bin', axis=1)
y_train = train[:35000]['fav_novel_cnt_bin']
x_test = train[35000:].drop('fav_novel_cnt_bin', axis=1)
y_test = train[35000:]['fav_novel_cnt_bin']

print(len(x_train))
print(len(x_test))

35000
5000


## 勾配ブースティング


In [7]:
#勾配ブースティング
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.3, random_state=0, stratify=y_train)
categorical_features = ['biggenre', 'genre']

import lightgbm as lgb
lgb_train = lgb.Dataset(x_train, y_train, categorical_feature=categorical_features)
lgb_eval = lgb.Dataset(x_valid, y_valid, categorical_feature=categorical_features)

params = {
    'objective': 'binary'
}

model = lgb.train(
    params,
    lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    verbose_eval=10,
    num_boost_round=1000,
    early_stopping_rounds=10
)
pred = model.predict(x_test, num_iteration=model.best_iteration)

C:\Users\ngnls\AppData\Local\Programs\Python\Python39\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\ngnls\AppData\Local\Programs\Python\Python39\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\ngnls\AppData\Local\Programs\Python\Python39\lib\site-packages\lightgbm\engine.py:240: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: title

In [17]:

pred_to_fav = lambda x: x*10//2

# y_pred = y_test.copy()
# y_pred['fav_novel_cnt_bin'] = list(map(pred_to_fav, pred))
# y_pred.drop(index='fav_novel_cnt_bin', axis=0)
# y_pred['fav_novel_cnt_bin']=pd.to_numeric(y_pred['fav_novel_cnt_bin'])

y_pred = pred_to_fav(pred)
y_test_array = y_test.values


#正答率の確認
from sklearn.metrics import accuracy_score
print('accuracy = ', accuracy_score(y_true=y_test_array, y_pred=y_pred))


accuracy =  0.1326
